In [ ]:
%load_ext autoreload 
%autoreload 2

In [ ]:
import json
from privacypacking.utils.utils import load_logs, global_metrics
import pandas as pd
from experiments.ray.analysis import load_ray_experiment, load_latest_ray_experiment, load_latest_scheduling_results, load_latest_scheduling_results
import plotly.express as px
from privacypacking.budget.curves import  LaplaceCurve, GaussianCurve, SubsampledGaussianCurve
from privacypacking.budget import Budget, Task, Block
from privacypacking.schedulers.metrics import OverflowRelevance, FlatRelevance
from privacypacking.budget.block_selection import RandomBlocks
from privacypacking.utils.plot import plot_budgets
import yaml

In [ ]:
block = Budget.from_epsilon_delta(epsilon=10, delta=1e-5)

In [ ]:
t1 = GaussianCurve(sigma=3) + LaplaceCurve(laplace_noise=1)
t2 = GaussianCurve(sigma=2) + LaplaceCurve(laplace_noise=2)
plot_budgets([t1, t2, block])

In [ ]:
print(yaml.dump(t1.epsilons))
print()
print(yaml.dump(t2.epsilons))

In [ ]:
plot_budgets([
    t1.normalize_by(block),
    t2.normalize_by(block),
])

In [ ]:
plot_budgets([
    (t1 * 10).approx_epsilon_bound(delta=1e-5),
    (t2 * 10).approx_epsilon_bound(delta=1e-5),
])

In [ ]:
task1 = Task(id=1, profit=1, block_selection_policy=RandomBlocks, n_blocks=1)
task1.budget_per_block[1] = t1

task2 = Task(id=2, profit=1, block_selection_policy=RandomBlocks, n_blocks=1)
task2.budget_per_block[1] = t2

block1 = Block(id=1, budget=block)



In [ ]:
task1.budget_per_block[1]

In [ ]:
task2.budget_per_block[1]

In [ ]:
OverflowRelevance.apply(task1, blocks={1:block1}, tasks=[task1, task2]*100)

In [ ]:
OverflowRelevance.apply(task2, blocks={1:block1}, tasks=[task1, task2]*100)

In [ ]:
FlatRelevance.apply(task1, blocks={1:block1}, tasks=[task1, task2]*100)

In [ ]:
FlatRelevance.apply(task2, blocks={1:block1}, tasks=[task1, task2]*100)

In [ ]:
df = load_latest_ray_experiment()

In [ ]:
px.line(df.sort_values('n_tasks'), x="n_tasks", y="n_allocated_tasks", color="scheduler_metric", title="Single DPF killer", facet_col="block_selecting_policy")

In [ ]:
df = load_latest_ray_experiment()
px.line(df.sort_values('n_tasks'), x="n_tasks", y="n_allocated_tasks", color="scheduler_metric", title="Single DPF killer", facet_col="block_selecting_policy")

In [ ]:
df = load_latest_ray_experiment()
px.line(df.sort_values('n_tasks'), x="n_tasks", y="n_allocated_tasks", color="scheduler_metric", title="Single DPF killer", facet_col="block_selecting_policy")

In [ ]:
df = load_latest_ray_experiment()
px.line(df.sort_values('n_tasks'), x="n_tasks", y="n_allocated_tasks", color="scheduler_metric", title="Single DPF killer", facet_col="scheduler")

In [ ]:
t1 = GaussianCurve(sigma=5) + LaplaceCurve(laplace_noise=2)
t2 = GaussianCurve(sigma=3) + LaplaceCurve(laplace_noise=5)
plot_budgets([t1, t2, block])

In [ ]:
plot_budgets([
    (t1 * 20).approx_epsilon_bound(delta=1e-5),
    (t2 * 20).approx_epsilon_bound(delta=1e-5),
])

In [ ]:
print(yaml.dump(t1.epsilons))
print()
print(yaml.dump(t2.epsilons))

In [ ]:
task1 = Task(id=1, profit=1, block_selection_policy=RandomBlocks, n_blocks=1)
task1.budget_per_block[1] = t1

task2 = Task(id=2, profit=1, block_selection_policy=RandomBlocks, n_blocks=1)
task2.budget_per_block[1] = t2

block1 = Block(id=1, budget=block)

In [ ]:
for t in [task1, task2]:
    print(OverflowRelevance.apply(t, blocks={1:block1}, tasks=[task1, task2]*100))
    print(FlatRelevance.apply(t, blocks={1:block1}, tasks=[task1, task2]*100))

In [ ]:
df = load_latest_ray_experiment()
px.line(df.sort_values('n_tasks'), x="n_tasks", y="n_allocated_tasks", color="scheduler_metric", title="Single DPF killer", facet_col="scheduler")

In [ ]:
df = load_latest_ray_experiment()
def func(row):
    row["scheduler_metric"] = row["scheduler"] if row["scheduler"] == "simplex" else row["scheduler_metric"]
    return row
df = df.apply(
    func, axis="columns" 

)

px.line(df.sort_values('n_tasks'), x="n_tasks", y="n_allocated_tasks", color="scheduler_metric", title="Single DPF killer")

In [ ]:
df

In [ ]:
t1 = GaussianCurve(sigma=1)
t2 = SubsampledGaussianCurve(sigma=1, steps=100, sampling_probability=0.01)
plot_budgets([t1, t2, block], log_y=True)

In [ ]:
from collections import defaultdict

In [ ]:
data = defaultdict(list)
curves = [t1.approx_epsilon_bound(delta=1e-5), t2.approx_epsilon_bound(delta=1e-5)]
for curve_id, curve in enumerate(curves):
    for alpha, epsilon in zip(curve.alphas, curve.epsilons):
        data["alpha"].append(alpha)
        data["epsilon"].append(epsilon)
        data["id"].append(curve_id)
df = pd.DataFrame(data)
df

In [ ]:
df.to_csv("/home/pierre/privacypacking-paper/results/single_block_killer/standard_epsilon_bound.csv", index=False)

In [ ]:
plot_budgets([
    (t1 * 10).approx_epsilon_bound(delta=1e-5),
    (t2 * 10).approx_epsilon_bound(delta=1e-5),
], log_y=True)

In [ ]:
print(yaml.dump(t1.epsilons))
print()
print(yaml.dump(list(map(lambda x : float(x),t2.epsilons))))

In [ ]:
df = load_latest_ray_experiment()
def func(row):
    row["scheduler_metric"] = row["scheduler"] if row["scheduler"] == "simplex" else row["scheduler_metric"]
    return row
df = df.apply(
    func, axis="columns" 

)

px.line(df.sort_values('n_tasks'), x="n_tasks", y="n_allocated_tasks", color="scheduler_metric", title="Single DPF killer")

In [ ]:
df = load_latest_scheduling_results()

In [ ]:
df

In [ ]:
px.bar(
#     df,
    df.query("allocated"),
    x="block",
    y="epsilon",
#     color="log_id",
    color="id",
    # barmode="group",
    pattern_shape="n_blocks",
    facet_col="totalblocks_scheduler_selection",
#     animation_frame="id"
)
